#Import library for Environment

In [ ]:
import scipy
from scipy import special
import numpy as np
from scipy.special import lambertw
import math

dtype = np.float32

#Import library for DDPG Agent

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
from enviroment import Env_cellular as env
import matplotlib.pyplot as plt

# Create environment for cellular BS

In [ ]:
class Env_cellular():
    def __init__(self,    MAX_EP_STEPS, s_dim, location_vector, location_GF,K, Pn,fading_n, fading_0):
        self.emax = 0.1  # battery capacity
        #self.P0 = P0
        self.K = K
        self.T =1
        self.eta =0.7
        self.Pn = Pn # grant-based user's power
        self.Pmax = 0.1


        BW = 10**6 # 10MHz
        sigma2_dbm = -170 + 10 * np.log10(BW) #  Thermal noise in dBm
        #sigma2_dbm = -94
        self.noise = 10 ** ((sigma2_dbm - 30) / 10)
        self.Pn = self.Pn

        #self.P0n = 10 #grant free user's power
        self.s_dim = s_dim

        self.MAX_EP_STEPS = MAX_EP_STEPS


        distance_GF = np.sqrt(np.sum((location_vector-location_GF)**2, axis=1))
        distance_GB = np.sqrt(np.sum((location_vector)**2, axis=1))

        distance = np.matrix.transpose(np.array([distance_GF, distance_GB]))
        distance = np.maximum(distance, np.ones((self.K,2)))
        PL_alpha = 3
        PL = 1/distance**PL_alpha/(10**3.17)
        self.hn = np.multiply( PL, fading_n)
        distance_GF0 = np.sqrt(np.sum( location_GF ** 2, axis=1))
        distance0 = np.maximum(distance_GF0, 1)
        PL0 = 1/(distance0 ** PL_alpha)/(10**3.17)
        self.h0 = fading_0*PL0
        #print(f"{self.hn} and {self.h0}")

        self.channel_sequence = np.zeros(( self.MAX_EP_STEPS,2))
        for i in range(self.MAX_EP_STEPS):
            id_index = i % self.K
            self.channel_sequence[i,:] = self.hn[id_index,:]

################################################################################

    def step(self, action, state, j):
        hn = state[0,0]/self.noise
        hn0 = state[0,1]
        h0 = self.h0/self.noise #state[0,2]/self.noise
        En = state[0,-1]


        #print(uchannel)

        En_bar = action*min(self.emax-En, self.T*self.eta*self.Pn*hn0) - (1-action)*min(En,self.T*self.Pmax)

        #print(f"energy is {self.T*self.eta*self.Pn*hn0}")
        #print(f"en_bar is {En_bar}, and action is {action}")
        mu1 = self.eta*self.Pn*hn0*h0/(1+self.Pn*hn)
        mu2 = En_bar*h0/self.T/(1+self.Pn*hn)
        wx0 = np.real(lambertw(math.exp(-1)*(mu1-1), k=0))
        #print(f"{np.exp(wx0+1) } ")
        alphaxx = (mu1-mu2)/(math.exp(wx0+1) - 1 + mu1)
        alpha01 = 1 - (En+En_bar)/(self.T*self.eta*self.Pn*hn0)
        alpha02 = (self.T*self.eta*self.Pn*hn0-En_bar)\
                  /(self.T*self.eta*self.Pn*hn0 + self.T*self.Pmax)
        alphax2 = max(alpha01, alpha02)
        alphan = min(1, max(alphaxx,alphax2))
        #print(alphan)
        if En_bar == self.T * self.eta * self.Pn * hn0:
            P0n=0
            reward = 0  # remark in the paper
        elif alphan ==0:#<= 0.00000001:
            P0n=0
            reward = 0
        else:
            P0n = (1-alphan)*self.eta*self.Pn*hn0/alphan - En_bar/alphan/self.T
            #print(f"power is {P0n}, en is {En}, enbar is {En_bar}, alpha is {alphan}")
            #print(f"power {P0n} divided by noise {self.noise} as {P0n/self.noise}")
            reward = alphan*np.log(1 + P0n*h0/(1 +self.Pn*hn))

        #print(f"power is {P0n}, en is {En}, enbar is {En_bar}, alpha is {alphan}")
        #print(f"signal power is {P0n*h0} and inter is {self.Pn*hn}")
        #print(f"noise is {self.noise}")
        #print(f"signal strentch={P0n} times {h0},   interference is {self.Pn * hn}")
        #print(f"Pn is {self.Pn} and hn is {hn}")
        #print(f"rate is {np.log(1 + P0n * h0 / (1 + self.Pn * hn))}")
        #print(f"signal strentch={P0n} times {h0}, noise is {self.noise}, interference is {self.Pn*hn}")
        #print(f"{j} iteration - reward is {reward}")
        #print(f"get {(1-alphan)*self.T * self.eta * self.Pn * hn0} and consumed {alphan*self.T*P0n}")
        #print(f"get {(1-alphan)}* {self.T} * {self.eta} * {self.Pn} * {hn0} ")

        #print(f"the first part of power{(1 - alphan)}*{self.eta * self.Pn * hn0 / alphan}")
        #print(f"the second part of power{- En_bar/alphan/self.T}")

        if math.isnan(reward):
            print(f"start---------------------------------------")
            print(f"action is{action} ")
            print(f"finallly alphan is {alphan}")
            print(f"end-----------------------------------")
            reward = 0
        #if reward <0:
        #    print(f"alpha is {alphan} and power is {P0n}, reward is {reward}, "
        #          f"bound is {1 - (En_bar)/(self.T*self.eta*self.Pn*hn0)}, enbar is {En_bar}, exist battery is {En}")


        batter_new = min(self.emax, En + En_bar)

        #print(f"{j} - Energy:  {En} minus {(alphan)*self.T*P0n} plus {(1-alphan)*self.T*self.eta*self.Pn*hn0}, remain {batter_new}, enbar is {En_bar} ")
        #print(f"Parameters are - alpha {alphan}, power is {P0n},   GB-GF channel is {hn0},    reward is {reward}")

        state_next = self.channel_sequence[(j+1) % self.K, :].tolist()
        #state_next.append(self.h0)
        state_next.append(batter_new)
        state_next = np.reshape(state_next, (1, self.s_dim))

        done=False

        return reward, state_next, done

################################################################################

    def step_greedy(self,   state, j):
        hn = state[0,0]/self.noise
        hn0 = state[0,1]
        h0 = self.h0/self.noise
        En = state[0,-1]
        #print(uchannel)

        alphan = min(1, En/self.T/self.Pmax)
        if alphan==0:
            reward = 0
        else:
            reward = alphan*np.log(1 + self.Pmax*h0/(1 +self.Pn*hn))
            #print(f"random alpha is {alphan}- signal strentch={self.Pmax} times {h0}, noise is {self.noise}, interference is {self.Pn*hn}")
            #print(f"{j}-iteration - energy is {En}-random reward is {reward}")


        if math.isnan(reward):
            print(f"alpha is {alphan}  ")
            print(f"{En}")
        batter_new = min(self.emax, En -alphan*self.T*self.Pmax +(1-alphan)*self.T*self.eta*self.Pn*hn0)

        #print(f"{j} random - Energy:  {En} minus {(alphan)*self.T*self.Pmax} plus {(1-alphan)*self.T*self.eta*self.Pn*hn0}, remain {batter_new} ")
        #print(f"Parameters are - alpha {alphan}, power is {self.Pmax},   GB-GF channel is {hn0},    reward is {reward}")

        state_next = self.channel_sequence[(j+1) % self.K, :].tolist()
        #state_next.append(self.h0)
        state_next.append(batter_new)
        state_next = np.reshape(state_next, (1, self.s_dim))

        done=False

        return reward, state_next, done

    def step_random(self,   state, j):
        hn = state[0,0]/self.noise
        hn0 = state[0,1]
        h0 = self.h0/self.noise
        En = state[0,-1]
        #print(uchannel)

        alphan = np.random.uniform(0, min(1, En/self.T/self.Pmax))
        if alphan==0:
            reward = 0
        else:
            reward = alphan*np.log(1 + self.Pmax*h0/(1 +self.Pn*hn))
            #print(f"random alpha is {alphan}- signal strentch={self.Pmax} times {h0}, noise is {self.noise}, interference is {self.Pn*hn}")
            #print(f"{j}-iteration - energy is {En}-random reward is {reward}")


        if math.isnan(reward):
            print(f"alpha is {alphan}  ")
            print(f"{En}")
        batter_new = min(self.emax, En -alphan*self.T*self.Pmax +(1-alphan)*self.T*self.eta*self.Pn*hn0)

        #print(f"{j} random - Energy:  {En} minus {(alphan)*self.T*self.Pmax} plus {(1-alphan)*self.T*self.eta*self.Pn*hn0}, remain {batter_new} ")
        #print(f"Parameters are - alpha {alphan}, power is {self.Pmax},   GB-GF channel is {hn0},    reward is {reward}")

        state_next = self.channel_sequence[(j+1) % self.K, :].tolist()
        #state_next.append(self.h0)
        state_next.append(batter_new)
        state_next = np.reshape(state_next, (1, self.s_dim))

        done=False

        return reward, state_next, done

    def reset(self):
        batter_ini = self.emax
        return batter_ini

# Define DDPG Agent

In [ ]:

#####################  hyper parameters  ####################
MAX_EP_STEPS = 100
LR_A = 0.002    # learning rate for actor
LR_C = 0.004    # learning rate for critic
GAMMA = 0.9     # reward discount
TAU = 0.01      # soft replacement
MEMORY_CAPACITY = 10000
BATCH_SIZE = 32

#####################  INIT  ####################
INIT = 0
#### INIT:
# RANDOM LOCATION WITH FADING K: 0
# TWO USER CASE                : 1
# SPECIAL CASE                 : 2
if INIT == 0:         # RANDOM LOCATION WITH FADING K #
  ct=500              # 
  MAX_EPISODES =10    #

else if INIT == 1:    # TWO USER CASE #
  Pn = 1              #
  K=2                 #
  MAX_EPISODES = 400  #
  MAX_EP_STEPS = 100  #
else:                 # SPECIAL CASE #
  Pn = 1
  K=10 # the number of grant based users
  MAX_EPISODES = 400

###############################  DDPG  ####################################
class DDPG(object):
  def __init__(self, a_dim, s_dim, a_bound,):
    self.memory = np.zeros((MEMORY_CAPACITY, s_dim * 2 + a_dim + 1), dtype=np.float32)
    self.pointer = 0
    self.sess = tf.Session()


    self.a_dim, self.s_dim, self.a_bound = a_dim, s_dim, a_bound,
    self.S = tf.placeholder(tf.float32, [None, s_dim], 's')
    self.S_ = tf.placeholder(tf.float32, [None, s_dim], 's_')
    self.R = tf.placeholder(tf.float32, [None, 1], 'r')

    self.a = self._build_a(self.S,)
    q = self._build_c(self.S, self.a, )
    a_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Actor')
    c_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Critic')
    ema = tf.train.ExponentialMovingAverage(decay=1 - TAU)          # soft replacement

    def ema_getter(getter, name, *args, **kwargs):
      return ema.average(getter(name, *args, **kwargs))

    target_update = [ema.apply(a_params), ema.apply(c_params)]      # soft update operation
    a_ = self._build_a(self.S_, reuse=True, custom_getter=ema_getter)   # replaced target parameters
    q_ = self._build_c(self.S_, a_, reuse=True, custom_getter=ema_getter)

    a_loss = - tf.reduce_mean(q)  # maximize the q
    self.atrain = tf.train.AdamOptimizer(LR_A).minimize(a_loss, var_list=a_params)

    with tf.control_dependencies(target_update):    # soft replacement happened at here
      q_target = self.R + GAMMA * q_
      td_error = tf.losses.mean_squared_error(labels=q_target, predictions=q)
      self.ctrain = tf.train.AdamOptimizer(LR_C).minimize(td_error, var_list=c_params)

    self.sess.run(tf.global_variables_initializer())

  def choose_action(self, s):
    return self.sess.run(self.a, {self.S: s })[0]

  def learn(self):
    indices = np.random.choice(min(MEMORY_CAPACITY,self.pointer), size=BATCH_SIZE)
    bt = self.memory[indices, :]
    bs = bt[:, :self.s_dim]
    ba = bt[:, self.s_dim: self.s_dim + self.a_dim]
    br = bt[:, -self.s_dim - 1: -self.s_dim]
    bs_ = bt[:, -self.s_dim:]

    self.sess.run(self.atrain, {self.S: bs})
    self.sess.run(self.ctrain, {self.S: bs, self.a: ba, self.R: br, self.S_: bs_})

  def store_transition(self, s, a, r, s_):
      r = np.reshape(r,(1,1))
      a = np.reshape(a,(1,1))
      #print(f"state is {s}, action is {a}, reward is {r}, next state is {s_}")
      transition = np.hstack((s, a, r, s_))
      index = self.pointer % MEMORY_CAPACITY  # replace the old memory with new memory
      self.memory[index, :] = transition
      self.pointer += 1

  # Actor network 
  # s: state
  # reuse:
  # custom_getter:
  def _build_a(self, s, reuse=None, custom_getter=None):
    trainable = True if reuse is None else False
    with tf.variable_scope('Actor', reuse=reuse, custom_getter=custom_getter):
      net = tf.layers.dense(s, 64, activation=tf.nn.relu, name='l1', trainable=trainable)  # fully connected
      a2 = tf.layers.dense(net, 64, activation=tf.nn.tanh, name='l2', trainable=trainable) # fully connected
      #a3 = tf.layers.dense(a2, 30, activation=tf.nn.tanh, name='l3', trainable=trainable) # fully connected

      a = tf.layers.dense(a2, self.a_dim, activation=tf.nn.tanh, name='a', trainable=trainable)
      return tf.multiply(a, self.a_bound, name='scaled_a')

  # Critic network
  # s: state
  # a: action
  # reuse:
  # custom_getter: 
  def _build_c(self, s, a, reuse=None, custom_getter=None):
    trainable = True if reuse is None else False
    with tf.variable_scope('Critic', reuse=reuse, custom_getter=custom_getter):
      n_l1 = 64
      w1_s = tf.get_variable('w1_s', [self.s_dim, n_l1], trainable=trainable)
      w1_a = tf.get_variable('w1_a', [self.a_dim, n_l1], trainable=trainable)
      b1 = tf.get_variable('b1', [1, n_l1], trainable=trainable)
      net = tf.nn.relu(tf.matmul(s, w1_s) + tf.matmul(a, w1_a) + b1)
      net2 = tf.layers.dense(net, 64, activation=tf.nn.relu, name='lx2', trainable=trainable)
      #net3 = tf.layers.dense(net2, 30, activation=tf.nn.relu, name='lx3', trainable=trainable)

      #not sure about this part
      return tf.layers.dense(net2, 1, trainable=trainable)  # Q(s,a)


#**Random location with fading K: TRAINING**

In [ ]:
###############################  training  ####################################
s_dim = 3# dimsion of states
a_dim = 1# dimension of action
a_bound = 1 #bound of action
state_am = 10000


#GB_power_dBm = np.array([0,  10,  20,   30])
K_range = np.array([2,4,6,8, 10])
rate_DDPG = []
rate_greedy = []
rate_random = []
for k in range(len(K_range)):
  K = K_range[k]
  ratect_DDPG = 0
  ratect_greedy = 0
  ratect_random = 0
  for mct in range(ct):
    var = 1  # control exploration
    ddpg = DDPG(a_dim, s_dim, a_bound)

    locationspace = np.linspace(1, 1000, num=K)
    location_vector = np.zeros((K, 2))
    location_vector[:, 1] = locationspace
    location_GF = np.array([[1,1]])# np.ones((1, 2))

    ##### fading for GB user
    hnx1 = np.random.randn(K, 2)
    hnx2 = np.random.randn(K, 2)
    fading_n = hnx1 ** 2 + hnx2 ** 2
    #### fading for GF user
    h0x1 = np.random.randn(1, 1)
    h0x2 = np.random.randn(1, 1)
    fading_0 =  h0x1[0,0] ** 2 + h0x2[0,0] ** 2

    Pn = 10**((30-30)/10)
    myenv = env( MAX_EP_STEPS, s_dim, location_vector,location_GF,K,Pn, fading_n, fading_0)

    ratek_DDPG = 0
    ratek_greedy = 0
    ratek_random = 0
    for i in range(MAX_EPISODES):
      batter_ini = myenv.reset()
      s = myenv.channel_sequence[i % myenv.K, :].tolist()  # the current GB user, 2 element [GB-GF, GB-BS]
      # s.append(myenv.h0)
      s.append(batter_ini)
      s = np.reshape(s, (1, s_dim))
      s = s * state_am  # amplify the state
      s_greedy = s
      s_random = s
      reward_dpgg_vector =[]
      reward_greedy_vector = []
      reward_random_vector = []
      #print(s[0,0:2])
      for j in range(MAX_EP_STEPS):

        # Add exploration noise
        a = ddpg.choose_action(s)
        a = np.clip(np.random.normal(a, var), 0, 1)    # add randomness to action selection for exploration
        #print(myenv.location)
        r, s_, done = myenv.step(a, s / state_am, j)
        s_ = s_ * state_am
        #print(f"{j} state is {s}, action is {a}, reward is {r}, next state is {s_}")
        ddpg.store_transition(s, a, r, s_)
        if var>0.05:
          var *= .9998  # decay the action randomness
        ddpg.learn()
        s = s_
        reward_dpgg_vector.append(r)

        ##### greedy
        r_greedy, s_next_greedy, done = myenv.step_greedy(s_greedy/state_am, j)
        s_greedy = s_next_greedy*state_am
        reward_greedy_vector.append(r_greedy)

        ##### random
        r_random, s_next_random, done = myenv.step_random(s_random/state_am, j)
        s_random = s_next_random*state_am
        reward_random_vector.append(r_random)



        if j == MAX_EP_STEPS-1:
          #print(f"Iteration., {k}--{mct},  Episode:, {i},  Reward: {sum(reward_dpgg_vector)/MAX_EP_STEPS},")
          #      f" Reward Greedy:   {sum(reward_greedy_vector)/MAX_EP_STEPS},"
          #      f" Reward random:   { sum(reward_random_vector)/MAX_EP_STEPS}, Explore: {var} ")
          break
      ratek_DDPG = sum(reward_dpgg_vector)/MAX_EP_STEPS
      ratek_greedy = sum(reward_greedy_vector)/MAX_EP_STEPS
      ratek_random = sum(reward_random_vector)/MAX_EP_STEPS

    tf.keras.backend.clear_session()
    ratect_DDPG +=ratek_DDPG
    ratect_greedy +=ratek_greedy
    ratect_random += ratek_random
    print(ratek_DDPG)
  print(f"Iteration., {k}--{mct},  Episode:, {i},  Reward: {ratect_DDPG/ct },")
  rate_DDPG.append(ratect_DDPG/ct )
  rate_greedy.append(ratect_greedy /ct )
  rate_random.append(ratect_random  /ct)

print(f"rate_greedy is {rate_greedy} and rate_random is {rate_random}")
#GB_power_label = ['0',  '10',  '20', '30']
GB_power_label = ['0', '5', '10', '15', '20', '25', '30']
K_rangex =  ['2', '4', '6', '8', '10']
plt.plot(K_rangex,rate_DDPG, "^-", label='DDPG: rewards')
plt.plot(K_rangex,rate_greedy, "+:", label='Greedy: rewards')
plt.plot(K_rangex,rate_random, "o--", label='Random: rewards')
plt.xlabel("The Number of Primary Users (K)")
plt.ylabel("  Average  Data Rate (NPCU)")
plt.legend()
plt.show()


''' Save final results'''
#np.savez_compressed('data/dataK', rate_DDPG=rate_DDPG, rate_greedy=rate_greedy, rate_random=rate_random)

#**Two user case**

In [ ]:

s_dim = 3# dimsion of states
a_dim = 1# dimension of action
a_bound = 1 #bound of action
state_am = 1000


location_vector = np.array([[0, 1],[0,1000]]) #locations of GB users


location_GF = np.array([[1,1]])# np.ones((1, 2))

##### fading for GB user
hnx1 = np.random.randn(K, 2)
hnx2 = np.random.randn(K, 2)
fading_n = 1#hnx1 ** 2 + hnx2 ** 2
#### fading for GF user
h0x1 = np.random.randn(1, 1)
h0x2 = np.random.randn(1, 1)
fading_0 = 1#h0x1[0,0] ** 2 + h0x2[0,0] ** 2


myenv = env( MAX_EP_STEPS, s_dim, location_vector,location_GF,K,Pn, fading_n, fading_0)
#myenv = env(P0, MAX_EP_STEPS, s_dim, location_vector,location_GF,K)

#myenv = env(P0,MAX_EP_STEPS,s_dim)

ddpg = DDPG(a_dim, s_dim, a_bound)

var = 1  # control exploration
t1 = time.time()
ep_rewardall = []
ep_rewardall_greedy = []
ep_rewardall_random = []
for i in range(MAX_EPISODES):
  batter_ini = myenv.reset()
  s = myenv.channel_sequence[i%myenv.K,:].tolist() #the current GB user, 2 element [GB-GF, GB-BS]
  #s.append(myenv.h0)
  s.append(batter_ini)
  s = np.reshape(s,(1,s_dim))
  s = s*state_am #amplify the state
  s_greedy = s
  s_random = s
  #print(s[0,0:2])
  ep_reward = 0
  ep_reward_random = 0
  ep_reward_greedy = 0
  s_traj = []
  s_traj_random = []
  s_traj_greedy = []
  for j in range(MAX_EP_STEPS):

    # Add exploration noise
    a = ddpg.choose_action(s)
    #print(s)
    a = np.clip(np.random.normal(a, var), 0, 1)    # add randomness to action selection for exploration
    #print(myenv.location)
    r, s_, done = myenv.step(a,s/state_am,j)
    s_ = s_*state_am
    s_traj.append(s_)
    ddpg.store_transition(s, a, r, s_)

    if var > 0.1:
      var *= .9998  # decay the action randomness
    ddpg.learn()

    s = s_
    ep_reward += r

    ##### greedy
    r_greedy, s_next_greedy, done = myenv.step_greedy(s_greedy/state_am, j)
    s_traj_greedy.append(s_next_greedy)
    s_greedy = s_next_greedy*state_am
    ep_reward_greedy += r_greedy

    ##### random
    r_random, s_next_random, done = myenv.step_random(s_random/state_am, j)
    s_traj_random.append(s_next_random)
    s_random = s_next_random*state_am
    ep_reward_random += r_random


    if j == MAX_EP_STEPS-1:
      #print(f"Episode: {i}, reward is {ep_reward}, and Explore is {var}")
      print('Episode:', i, ' Reward: %i' % int(ep_reward),' Reward Greedy: %i' % int(ep_reward_greedy),' Reward random: %i' % int(ep_reward_random), 'Explore: %.2f' % var )
      #print(myenv.location)
      # if ep_reward > -300:RENDER = True
      break
  ep_reward = np.reshape(ep_reward/MAX_EP_STEPS, (1,))
  ep_rewardall.append(ep_reward)

  ep_reward_greedy = np.reshape(ep_reward_greedy/MAX_EP_STEPS, (1,))
  ep_rewardall_greedy.append(ep_reward_greedy)

  ep_reward_random = np.reshape(ep_reward_random/MAX_EP_STEPS, (1,))
  ep_rewardall_random.append(ep_reward_random)

#print(s_)
print('Running time: ', time.time() - t1)

print(f"{myenv.hn} and {myenv.h0}")
plt.plot(ep_rewardall, "^-", label='DDPG: rewards')
plt.plot(ep_rewardall_greedy, "+:", label='Greedy: rewards')
plt.plot(ep_rewardall_random, "o--", label='Random: rewards')
plt.xlabel("Episode")
plt.ylabel(" Epsiodic Reward -  Data Rate (NPCU)")
plt.legend()
plt.show()

''' Save final results'''
#np.savez_compressed('data/data_two_user', ep_rewardall=ep_rewardall, ep_rewardall_greedy=ep_rewardall_greedy, ep_rewardall_random=ep_rewardall_random)

#**Special case**

In [ ]:
s_dim = 3# dimsion of states
a_dim = 1# dimension of action
a_bound = 1 #bound of action
state_am = 10000


locationspace = np.linspace(1,1000, num=K)
location_vector = np.zeros((K, 2))
location_vector[:,1] = locationspace


location_GF = np.array([[1,1]])# np.ones((1, 2))

##### fading for GB user
hnx1 = np.random.randn(K, 2)
hnx2 = np.random.randn(K, 2)
fading_n = hnx1 ** 2 + hnx2 ** 2
#### fading for GF user
h0x1 = np.random.randn(1, 1)
h0x2 = np.random.randn(1, 1)
fading_0 = h0x1[0,0] ** 2 + h0x2[0,0] ** 2


myenv = env(  MAX_EP_STEPS, s_dim, location_vector,location_GF,K,Pn, fading_n, fading_0)
#myenv = env(P0, MAX_EP_STEPS, s_dim, location_vector,location_GF,K)

#myenv = env(P0,MAX_EP_STEPS,s_dim)

ddpg = DDPG(a_dim, s_dim, a_bound)

var = 1  # control exploration
t1 = time.time()
ep_rewardall = []
ep_rewardall_greedy = []
ep_rewardall_random = []
for i in range(MAX_EPISODES):
  batter_ini = myenv.reset()
  s = myenv.channel_sequence[i%myenv.K,:].tolist()
  #s.append(myenv.h0)
  s.append(batter_ini)
  s = np.reshape(s,(1,s_dim))
  s = s*state_am #amplify the state
  s_greedy = s
  s_random = s
  #print(s[0,0:2])
  ep_reward = 0
  ep_reward_random = 0
  ep_reward_greedy = 0
  for j in range(MAX_EP_STEPS):

    # Add exploration noise
    a = ddpg.choose_action(s)
    a = np.clip(np.random.normal(a, var), 0, 1)    # add randomness to action selection for exploration
    r, s_, done = myenv.step(a,s/state_am,j)
    s_ = s_ * state_am
    ddpg.store_transition(s, a, r, s_)
    if var >0.1:
      var *= .9998    # decay the action randomness

    ddpg.learn()
    s = s_
    ep_reward += r

    ##### greedy
    r_greedy, s_next_greedy, done = myenv.step_greedy(s_greedy/state_am, j)
    s_greedy = s_next_greedy*state_am
    ep_reward_greedy += r_greedy

    ##### random
    r_random, s_next_random, done = myenv.step_random(s_random/state_am, j)
    s_random = s_next_random*state_am
    ep_reward_random += r_random


    if j == MAX_EP_STEPS-1:
      #print(f"Episode: {i}, reward is {ep_reward}, and Explore is {var}")
      print('Episode:', i, ' Reward: %i' % int(ep_reward),' Reward Greedy: %i' % int(ep_reward_greedy),' Reward random: %i' % int(ep_reward_random), 'Explore: %.2f' % var )
      #print(myenv.location)
      # if ep_reward > -300:RENDER = True
      break
  ep_reward = np.reshape(ep_reward/MAX_EP_STEPS, (1,))
  ep_rewardall.append(ep_reward)

  ep_reward_greedy = np.reshape(ep_reward_greedy/MAX_EP_STEPS, (1,))
  ep_rewardall_greedy.append(ep_reward_greedy)

  ep_reward_random = np.reshape(ep_reward_random/MAX_EP_STEPS, (1,))
  ep_rewardall_random.append(ep_reward_random)

#print(s_)
print('Running time: ', time.time() - t1)

print(f"{ep_reward}  ")
print(ep_rewardall)
plt.plot(ep_rewardall, "^-", label='DDPG: rewards')
plt.plot(ep_rewardall_greedy, "+:", label='Greedy: rewards')
plt.plot(ep_rewardall_random, "o--", label='Random: rewards')
plt.xlabel("Episode")
plt.ylabel(" Epsiodic Reward - Average Data Rate (NPCU)")
plt.legend( loc=3,  ncol=2)
plt.show()

''' Save final results'''
#np.savez_compressed('data_snapshot', ep_rewardall=ep_rewardall, ep_rewardall_greedy=ep_rewardall_greedy, ep_rewardall_random=ep_rewardall_random)